# Playground for cause and effect two sentences

You can use the playground to test the different evaluation functions. If you want to run the entire evaluation please use the respective .py script.

In [1]:
import os
import numpy as np
import openai
import pandas as pd
from tqdm import tqdm
from setups.ce_two_sentences_setup import CETwoSentences

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_KEY")

In [2]:
gpt3_model = "text-davinci-001"
#gpt3_model = "text-curie-001"
#gpt3_model = "text-babbage-001"
#gpt3_model = "text-ada-001"

In [3]:
# test run
def gpt3_response(prompt, model):
    response = openai.Completion.create(engine=model, prompt=prompt, max_tokens=20, temperature=0)
    return(response["choices"][0]["text"])

In [4]:
ce_2_sentences = CETwoSentences()

In [5]:
ce_2_sentences.generate_sequences_df()

,sequence,answer_cause,answer_effect,switched
0,The child hurt their knee. The child started c...,The child hurt their knee.,The child started crying.,False
1,The child started crying. The child hurt their...,The child hurt their knee.,The child started crying.,True
2,My car got dirty. I washed the car.,My car got dirty.,I washed the car.,False
3,I washed the car. My car got dirty.,My car got dirty.,I washed the car.,True
4,Someone fainted. Someone called 911.,Someone fainted.,Someone called 911.,False
...,...,...,...,...
97,Ted had a hangover. Ted went out drinking.,Ted went out drinking.,Ted had a hangover.,True
98,Alice ate the last cookie. Bob opened the jar ...,Alice ate the last cookie.,Bob opened the jar and was frustrated.,False
99,Bob opened the jar and was frustrated. Alice a...,Alice ate the last cookie.,Bob opened the jar and was frustrated.,True
100,Jimmy took the last piece of cake. Mom rinsed ...,Jimmy took the last piece of cake.,Mom rinsed the tray.,False


In [6]:
ce_2_sentences.save_sequences_df(filepath="data/bigbench_csvs/ce_two_sentences.csv")

### Zero shot

In [7]:
prompt_zs_cause = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="cause")
prompt_zs_effect = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="effect")

prompt_zs_cause_switched = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="cause")
prompt_zs_effect_switched = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="effect")

print(prompt_zs_cause + gpt3_response(prompt_zs_cause, gpt3_model) + "\n")
print(prompt_zs_effect + gpt3_response(prompt_zs_effect, gpt3_model) + "\n")
print(prompt_zs_cause_switched + gpt3_response(prompt_zs_cause_switched, gpt3_model) + "\n")
print(prompt_zs_effect_switched + gpt3_response(prompt_zs_effect_switched, gpt3_model) + "\n")

The child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:

The child started crying.

The child hurt their knee. The child started crying. Question: Which sentence is the effect of the other?  Answer by copying the sentence:

The child's knee was hurting.

The child started crying. The child hurt their knee. Question: Which sentence caused the other?  Answer by copying the sentence:

The child started crying.

The child started crying. The child hurt their knee. Question: Which sentence is the effect of the other?  Answer by copying the sentence:

The child started crying because they hurt their knee.



In [8]:
print(ce_2_sentences.generate_all_prompts_zero_shot(question='cause')[:3])
print(ce_2_sentences.generate_all_prompts_zero_shot(question='effect')[:3])

['The child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:', 'The child started crying. The child hurt their knee. Question: Which sentence caused the other?  Answer by copying the sentence:', 'My car got dirty. I washed the car. Question: Which sentence caused the other?  Answer by copying the sentence:']
['The child hurt their knee. The child started crying. Question: Which sentence is the effect of the other?  Answer by copying the sentence:', 'The child started crying. The child hurt their knee. Question: Which sentence is the effect of the other?  Answer by copying the sentence:', 'My car got dirty. I washed the car. Question: Which sentence is the effect of the other?  Answer by copying the sentence:']


### One shot

In [9]:
prompt_os_cause = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=False, question="cause")
prompt_os_effect = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=False, question="effect")

prompt_os_cause_switched = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=True, switched_shot=True, question="cause")
prompt_os_effect_switched = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=True, switched_shot=True, question="effect")

prompt_os_cause_switched_nss = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=True, switched_shot=False, question="cause")
prompt_os_effect_switched_nss = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=True, switched_shot=False, question="effect")

prompt_os_cause_ss = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=False, switched_shot=True, question="cause")
prompt_os_effect_ss = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=False, switched_shot=True, question="effect")

print("normal")
print(prompt_os_cause + gpt3_response(prompt_os_cause, gpt3_model) + "\n")
print(prompt_os_effect + gpt3_response(prompt_os_effect, gpt3_model) + "\n")

print("both switched")
print(prompt_os_cause_switched + gpt3_response(prompt_os_cause_switched, gpt3_model) + "\n")
print(prompt_os_effect_switched + gpt3_response(prompt_os_effect_switched, gpt3_model) + "\n")

print("prompt switched, shot normal")
print(prompt_os_cause_switched_nss + gpt3_response(prompt_os_cause_switched_nss, gpt3_model) + "\n")
print(prompt_os_effect_switched_nss + gpt3_response(prompt_os_effect_switched_nss, gpt3_model) + "\n")

print("prompt normal, shot switched")
print(prompt_os_cause_ss + gpt3_response(prompt_os_cause_ss, gpt3_model) + "\n")
print(prompt_os_effect_ss + gpt3_response(prompt_os_effect_ss, gpt3_model) + "\n")

normal
A large object hit the Earth. The dinosaurs became extinct. Question: Which sentence caused the other? Answer by copying the sentence: A large object hit the Earth.
The child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:

The child hurt their knee.

A large object hit the Earth. The dinosaurs became extinct. Question: Which sentence is the effect of the other? Answer by copying the sentence: The dinosaurs became extinct.
The child hurt their knee. The child started crying. Question: Which sentence is the effect of the other?  Answer by copying the sentence: The child started crying.

both switched
The man sipped the milk and spit it out. The milk was left outside the fridge over night. Question: Which sentence caused the other? Answer by copying the sentence: The milk was left outside the fridge over night.
The child started crying. The child hurt their knee. Question: Which sentence caused the other?  Answ

In [10]:
print(ce_2_sentences.generate_all_prompts_one_shot(question='cause')[:2])
print("\n")
print(ce_2_sentences.generate_all_prompts_one_shot(question='effect')[:2])
print("\n")

['I flipped the switch. Light filled the room. Question: Which sentence caused the other? Answer by copying the sentence: I flipped the switch.\nThe child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:', 'The family went to the beach. The air conditioner broke. Question: Which sentence caused the other? Answer by copying the sentence: The air conditioner broke.\nThe child started crying. The child hurt their knee. Question: Which sentence caused the other?  Answer by copying the sentence:']


['The team won the game. The team won the championship. Question: Which sentence is the effect of the other? Answer by copying the sentence: The team won the championship.\nThe child hurt their knee. The child started crying. Question: Which sentence is the effect of the other?  Answer by copying the sentence:', "The man turned down the music volume. The man couldn't hear what the woman was saying. Question: Which sentence is 

In [11]:
print(ce_2_sentences.generate_all_prompts_one_shot_switched_shot(question='cause')[:2])
print("\n")
print(ce_2_sentences.generate_all_prompts_one_shot_switched_shot(question='effect')[:2])
print("\n")

["The dog bit me. I pulled the dog's tail. Question: Which sentence caused the other? Answer by copying the sentence: I pulled the dog's tail.\nThe child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:", 'The bread was dry. We made French toast. Question: Which sentence caused the other? Answer by copying the sentence: The bread was dry.\nThe child started crying. The child hurt their knee. Question: Which sentence caused the other?  Answer by copying the sentence:']


['Tomatoes grew on the vine. The insect pollinated the tomato flowers. Question: Which sentence is the effect of the other? Answer by copying the sentence: Tomatoes grew on the vine.\nThe child hurt their knee. The child started crying. Question: Which sentence is the effect of the other?  Answer by copying the sentence:', 'The air conditioner broke. The family went to the beach. Question: Which sentence is the effect of the other? Answer by copying t

### K=5 shot

In [12]:
prompt_ks_cause = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=False, question="cause")
prompt_ks_effect = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=False, question="effect")

prompt_ks_cause_switched = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=True, switched_shot=True, question="cause")
prompt_ks_effect_switched = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=True, switched_shot=True, question="effect")

prompt_ks_cause_switched_nss = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=True, switched_shot=False, question="cause")
prompt_ks_effect_switched_nss = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=True, switched_shot=False, question="effect")

prompt_ks_cause_ss = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=False, switched_shot=True, question="cause")
prompt_ks_effect_ss = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, switched=False, switched_shot=True, question="effect")

print("normal")
print(prompt_ks_cause + gpt3_response(prompt_ks_cause, gpt3_model) + "\n")
print(prompt_ks_effect + gpt3_response(prompt_ks_effect, gpt3_model) + "\n")

print("both switched")
print(prompt_ks_cause_switched + gpt3_response(prompt_ks_cause_switched, gpt3_model) + "\n")
print(prompt_ks_effect_switched + gpt3_response(prompt_ks_effect_switched, gpt3_model) + "\n")

print("prompt switched, shot normal")
print(prompt_ks_cause_switched_nss + gpt3_response(prompt_ks_cause_switched_nss, gpt3_model) + "\n")
print(prompt_ks_effect_switched_nss + gpt3_response(prompt_ks_effect_switched_nss, gpt3_model) + "\n")

print("prompt normal, shot switched")
print(prompt_ks_cause_ss + gpt3_response(prompt_ks_cause_ss, gpt3_model) + "\n")
print(prompt_ks_effect_ss + gpt3_response(prompt_ks_effect_ss, gpt3_model) + "\n")

normal
I flipped the switch. Light filled the room. Question: Which sentence caused the other? Answer by copying the sentence: I flipped the switch.
There was a loud noise. The boy lifted his eyes from the book. Question: Which sentence caused the other? Answer by copying the sentence: There was a loud noise.
It rained. The picnic was cancelled. Question: Which sentence caused the other? Answer by copying the sentence: It rained.
It started raining. The woman who was walking on the street opened her umbrella. Question: Which sentence caused the other? Answer by copying the sentence: It started raining.
A man accidently pushed the vase off a counter. The vase shattered on the floor. Question: Which sentence caused the other? Answer by copying the sentence: A man accidently pushed the vase off a counter.
The child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence: The child hurt their knee.

It rained. The picnic was can

In [13]:
print(ce_2_sentences.generate_all_prompts_k_shot(k=5, question='cause')[:2])
print("\n")
print(ce_2_sentences.generate_all_prompts_k_shot(k=5, question='effect')[:2])

["There was a loud noise. The boy lifted his eyes from the book. Question: Which sentence caused the other? Answer by copying the sentence: There was a loud noise.\nThe curtains were opened. Sunlight filled the room. Question: Which sentence caused the other? Answer by copying the sentence: The curtains were opened.\nThere was a snowstorm. School was cancelled. Question: Which sentence caused the other? Answer by copying the sentence: There was a snowstorm.\nThe driver ran a red light. The driver was pulled over and was fined. Question: Which sentence caused the other? Answer by copying the sentence: The driver ran a red light.\nJill won the lawsuit against Acme. Acme paid Jill's legal fees. Question: Which sentence caused the other? Answer by copying the sentence: Jill won the lawsuit against Acme.\nThe child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:", 'The team won the championship. The team won the game. Qu

In [14]:
print(ce_2_sentences.generate_all_prompts_k_shot_switched_shot(k=5, question='cause')[:2])
print("\n")
print(ce_2_sentences.generate_all_prompts_k_shot_switched_shot(k=5, question='effect')[:2])

['The man went to jail. The man robbed a gas station. Question: Which sentence caused the other? Answer by copying the sentence: The man robbed a gas station.\nSchool was cancelled. There was a snowstorm. Question: Which sentence caused the other? Answer by copying the sentence: There was a snowstorm.\nA man was covered in sticky red goo. A watermelon exploded. Question: Which sentence caused the other? Answer by copying the sentence: A watermelon exploded.\nMy friend celebrated with their family. My friend got a promotion. Question: Which sentence caused the other? Answer by copying the sentence: My friend got a promotion.\nThe ship capsized. The wind blew strong. Question: Which sentence caused the other? Answer by copying the sentence: The wind blew strong.\nThe child hurt their knee. The child started crying. Question: Which sentence caused the other?  Answer by copying the sentence:', "There was a loud noise. The boy lifted his eyes from the book. Question: Which sentence caused t